### Nuißl Sandra, 14.08.2023
## "Empirische Evaluation von ‚State Of The Art‘ Topic Modeling Ansätze am Beispiel von Produktreviews für die Entscheidungsunterstützung in Unternehmen"
### - Data Preperation -
<hr>

## **Inhaltsverzeichnis**

<ul>1. Aufbau des Jupyter Notebooks</ul>
<ul>2. Instalation und Imports</ul>
    <ul>
     <ul>2.1. Installationen</ul>
     <ul>2.2. Imports</ul>
    </ul>
<ul>3. Laden der Amazon Daten</ul>
    <ul>
     <ul>3.1. Entpacken der Zip Files</ul>
     <ul>3.2. Laden des Datensatzes in einen Data Frame</ul>
    </ul>
<ul>4. Allgemeine Datenbereinigung</ul>
    <ul>
     <ul>4.1. Selectieren und Zusammenführen der Spalten</ul>
     <ul>4.2. Nan-Values und doppelte Werte</ul>
     <ul>4.3. Löschen zu kurzer Reviews</ul>
     <ul>4.4. Selectieren der Jahre 2011 bis 2018</ul>
    </ul>
<ul>5. Bereinigung der Textdaten</ul>
    <ul>
     <ul>5.1. Lowercasing</ul>
     <ul>5.2. Lemmatisierung</ul>
     <ul>5.3. Stemming</ul>
     <ul>5.4. Entfernung von Satzzeichen</ul>
     <ul>5.5. Entfernung von Stopwords</ul>
     <ul>5.6. Entfernung von Zahlen</ul>
     <ul>5.7. Entfernung von nicht ASCII konformen Wörtern</ul>
     <ul>5.8. Selection relevanter Spalten für den Export</ul>
    </ul>
<ul>6. Export der Bereinigten Daten</ul>
<ul>7. Literaturverzeichnis</ul>
<hr>

## 1. Aufbau des Jupyter Notebooks
Dieses Jupyter Notebook ist Teil der Master Thesis "Empirische Evaluation von ‚State Of The Art‘ Topic Modeling Ansätze am Beispiel von Produktreviews für die Entscheidungsunterstützung in Unternehmen".
Dieses Jupyter Notebook ist für die Durchführung der Datenbereinigung zuständig, um einen fertigen Datensatz für die Topic Modelling Modelle zu erhalten.

Die theoretischen Inhalte zu diesem Notebook sind der schriftlichen Ausarbeitung dieser Masterarbeit unter dem Kapitel "4.3. Data Preperation" zu entnehmen.

## 2. Instalationen und Imports
### 2.1. Installationen

In [ ]:
# Instalationen
% pip install gzip
% pip install shutil
% pip install nltk

### 2.2. Imports

In [ ]:
# Imports und Initalisierungen
import datetime
import numpy as np
import pandas as pd
from pandas import json_normalize

# Progressbar
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
from tqdm.auto import tqdm
tqdm_notebook.pandas()

# Entpacken der Files
import gzip
import shutil

# Beschleunigung
import json
import joblib
from joblib import Parallel, delayed

# Text
import re
import string
from string import punctuation

# Natural Language Processing
import nltk
from nltk.stem.wordnet import WordNetLemmatizer             # Lemmatisierung zur Textdimensionsreduktion
nltk.download('averaged_perceptron_tagger')                 # Für POS-Tagging
from nltk import word_tokenize                              # Tokenisierung
from nltk import pos_tag                                    # Bestimmung der grammatikalischen Token
from nltk.stem.snowball import SnowballStemmer              # Stemmer zur Textdimensionsreduktion
from nltk.corpus import stopwords                           # Zur entfernung der Stopwords
nltk.download('stopwords')                                  # Herunterladen der Liste mit Stopwords
nltk.download('punkt')
nltk.download('wordnet')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 3. Laden der Amazon Daten

### 3.1. Entzippen der Json Files
Die Amazon Datensätze sind aufgrund der großen Datenmengen als Zip Dateien gespeichert. Um diese in das Jupyter Notebook einlesen zu können, müssen daher die JSON Files zuerst entpackt werden.

Zum Entpacken der Files wird im Folgenden "gzip" in Verbindung mit "shutil" verwendet.
Gzip ist ein Programm zur Daten Kompression (Free Software Foundation, 2022) während shutil ein Modul ist, welches diverse High-Level-Operationen zur Unterstützung beim Kopieren und Löschen von Dateien bietet (Python-Software-Foundation, 2023).
Durch deren Kombination werden zuerst zwei Files geöffnet. file_in beschreibt hier das gezippte JSON file und bei file_out handelt es sich um ein leeres JSON File, in welches die Daten aus file_in mithilfe der Funktion "copyfileobj" von shutil kopiert werden (Erick, 2018).

In [ ]:
# Entpacken der Datei und speichern in einem JSON File
# Quelle: https://stackoverflow.com/questions/31028815/how-to-unzip-gz-file-using-python
# Review Daten
with gzip.open('Sports_and_Outdoors_5.json.gz', 'rb') as file_in:
    with open('Sports_and_Outdoors_5.json', 'wb') as file_out:
        shutil.copyfileobj(file_in, file_out)

# Meta Daten
with gzip.open('meta_Sports_and_Outdoors.json.gz', 'rb') as file_in:
    with open('meta_Sports_and_Outdoors.json', 'wb') as file_out:
        shutil.copyfileobj(file_in, file_out)

### 3.2. Laden des Datensatzes in einen Data Frame

Nachdem die Datensätze entpackt wurden, können diese geladen in einen Data Frame gespeichert werden.

In [ ]:
# Review Daten
# Öffnen der Datei und speichern als Objekt in der Variable f
r = open('/content/drive/MyDrive/Masterarbeit_Nuißl/Archiv/Sports_and_Outdoors_5.json')

# Laden des JSON Files in das Jupyter Notebook mit Beschleunigung
dataset_review = Parallel(n_jobs=-1)(delayed(json.loads)(line) for line in tqdm(r, desc ="Status der geladenen Zeilen: ", total = 2839940))

# Normalisieren der Daten und Laden in einen Data Frame
review_df = json_normalize(dataset_review)

# Ausgabe
#pd.concat([review_df], axis=1, sort = False, keys = ["Review Datensatz: Sports and Outdoors"])

In [ ]:
# Meta Daten
# Öffnen der Datei und speichern als Objekt in der Variable f
m = open('/content/drive/MyDrive/Masterarbeit_Nuißl/Archiv/meta_Sports_and_Outdoors.json')

# Laden des JSON Files in das Jupyter Notebook mit Beschleunigung
dataset_meta = Parallel(n_jobs=-1)(delayed(json.loads)(line) for line in tqdm(m, desc ="Status der geladenen Zeilen: ", total = 962300))

# Normalisieren der Daten und Laden in einen Data Frame
meta_df = json_normalize(dataset_meta)

# Ausgabe
#pd.concat([meta_df], axis=1, sort = False, keys = ["Meta Datensatz: Sports and Outdoors"])

## 4. Allgemeine Datenbereinigungen

### 4.1. Selectieren und Zusammenführen der Spalten

Zu Beginn werden die Spalten, welche auf Grundlage der Datenanalyse als wichtig erachtet wurden selektiert. Im Anschluss werden die Reviews um die Informationen aus dem Meta Datensatz erweitert, wodurch sich folgender Data Frame ergibt:

In [ ]:
# Erstellen eines neuen DataFrames mit den relevanten Spalten
review_df = review_df[["reviewText", "asin", "overall", "unixReviewTime"]]
meta_df = meta_df[["asin", "brand", "title"]]

In [ ]:
# Ergänzung der Reviewdaten um die Meta Daten
merged_df = review_df.merge(meta_df, on="asin", how='left')
merged_df.head(10)

,reviewText,asin,overall,unixReviewTime,brand,title,text_len,year
0,What a spectacular tutu! Very slimming.,0000032034,5.0,2015-06-03,BubuBibi,Adult Ballet Tutu Yellow,6,2015
1,What the heck? Is this a tutu for nuns? I know...,0000032034,1.0,2015-04-01,BubuBibi,Adult Ballet Tutu Yellow,39,2015
2,Exactly what we were looking for!,0000032034,5.0,2015-01-13,BubuBibi,Adult Ballet Tutu Yellow,6,2015
3,I used this skirt for a Halloween costume and ...,0000032034,5.0,2014-12-23,BubuBibi,Adult Ballet Tutu Yellow,84,2014
4,This is thick enough that you can't see throug...,0000032034,4.0,2014-12-15,BubuBibi,Adult Ballet Tutu Yellow,28,2014
7,Arrived on time. Nice Atlas - Easy to read.,0899332757,5.0,2016-12-27,Garmin,Delorme New York State Atlas &amp; Gazetteer,9,2016
8,More of a road map than a useful topographical...,0899332757,3.0,2016-12-12,Garmin,Delorme New York State Atlas &amp; Gazetteer,37,2016
9,Comprehensive atlas. Very happy with how much ...,0899332757,5.0,2016-08-13,Garmin,Delorme New York State Atlas &amp; Gazetteer,32,2016
10,Delorme has always made the best book maps in ...,0899332757,5.0,2016-08-02,Garmin,Delorme New York State Atlas &amp; Gazetteer,14,2016
11,If you're interested in exploring any area of ...,0899332757,5.0,2016-08-01,Garmin,Delorme New York State Atlas &amp; Gazetteer,157,2016


### 4.2. Nan-Values und doppelte Werte

Um die Performence zu verbessern, werden alle Nan - Values sowie doppelten Werte genlöscht, da diese keinen Mehrwert liefern oder die Ergebnisse verfälschen könnten.

In [ ]:
# Bestimmung der Länge des Data Frames vor Drop
len_start = len(merged_df)

# Droppen der Nan - Values der reviewTexte
merged_df= merged_df.dropna(subset=['reviewText'])
merged_df.reset_index(inplace=True, drop=False)
merged_df= merged_df.drop("index", axis=1)

# Entfernung von Dublikaten
merged_df.drop_duplicates(inplace=True)
merged_df.reset_index(inplace=True, drop=False)
merged_df= merged_df.drop("index", axis=1)

# Bestimmung der Länge des Data Frames nach Drop
len_ende = len(merged_df)

# Ausgabe
print(f"Der Data Frame hat sich durch den Drop der Nan - Values und Doppelten Werte um {len_start - len_ende} Zeilen verringert.")

Der Data Frame hat sich durch den Drop der Nan - Values und Doppelten Werte um 193778 Zeilen verringert.


### 4.3. Löschen zu kurzer Reviews

Wie bereits in der Datenanalyse beschrieben, enthalten zu kurze Reviews keine relevanten Informationen für eine Topic Modelling Analyse. Mit dem Ziel den Datensatz etwas zu reduzieren und lediglich aussagekräftige Reviews zu verwenden, sollen im folgenden alle Bewertungen gelöscht werden, welche weniger als fünf Zeichen enthalten.

In [ ]:
# Funktion zur bestimmung der Länge der Texte pro "Dokument" / Zeile
def count_text_len(in_string):
    text_len = len(in_string.split())
    return text_len

In [ ]:
# Anwendung der Funktion zur Ermittlung der Länge (Wörter) der Reviews
merged_df["text_len"] = Parallel(n_jobs=-1)(delayed(count_text_len)(in_string) for in_string in tqdm(merged_df["reviewText"],
                                                                                                desc ="Status ermittelten Sprache pro Dokument: ",
                                                                                                total = len(merged_df["reviewText"])))

Status ermittelten Sprache pro Dokument:   0%|          | 0/2688115 [00:00<?, ?it/s]

In [ ]:
# Bestimmung der Länge des Data Frames vor Drop
len_start = len(merged_df)

# Beibehalten der Reviews mit 5 oder mehr Zeichen
merged_df = merged_df[(merged_df["text_len"] >= 5)]

# Bestimmung der Länge des Data Frames nach Drop
len_ende = len(merged_df)

# Ausgabe
print(f"Der Data Frame hat sich durch die Bereinigung der Reviews um {len_start - len_ende} Zeilen verringert.")

Der Data Frame hat sich durch die Bereinigung der Reviews um 397476 Zeilen verringert.


### 4.4. Selectieren der Jahre 2011 bis 2018

Der Amazon Datensatz für "Sport and Outdoors" enthält nicht über alle Jahr verteilt eine ähnliche Anzahl an Reviews. Aus diesem Grund sollen die Jahre 2011 bis 2018 selektiert werden, welche die höchste Anzahl an Bewertungen enthalten. Somit entfallen die Jahre 2000 bis 2010.

In [ ]:
# Umrechnung der unixReviewTime in einen Timestamp
# Quelle: https://www.adamsmith.haus/python/answers/how-to-convert-a-column-of-unix-times-in-a-pandas-dataframe-to-datetimes-in-python
merged_df["unixReviewTime"] = pd.to_datetime(merged_df["unixReviewTime"], unit='s')

# Aufsplitten in Jahre
merged_df['year'] = pd.DatetimeIndex(merged_df["unixReviewTime"]).year

In [ ]:
# Bestimmung der Länge des Data Frames vor Drop
len_start = len(merged_df)

# Beibehalten der Reviews mit 5 oder mehr Zeichen
merged_df = merged_df[(merged_df["year"] >= 2011)]

# Bestimmung der Länge des Data Frames nach Drop
len_ende = len(merged_df)

# Ausgabe
print(f"Der Data Frame hat sich durch die Selektion der Jahre um {len_start - len_ende} Zeilen verringert.")

Der Data Frame hat sich durch die Selektion der Jahre um 25998 Zeilen verringert.


## 5. Bereinigung der Textdaten

Mithilfe der vorangegangenen Bereinigung konnte eine kleine Dimensionsreduktion durchgeführt werden. Um die Texte weiter für die späteren Topic Modelling Modelle zu reduzieren und zu vereinheitlichen, werden Reihe von Techniken der Textverarbeitung herangezogen:

- Lowercasing
- Lemmatisierung
- Stemming
- Entfernung von Satzzeichen
- Entfernung von Stopwords
- Entfernung von Zahlen
- Entfernung von nicht ASCII konformen Wörtern
- Entfernung von http und Weblinks

Gestartet wird mit dem bereits reduzierten Datenesatz:


In [ ]:
# Bereinigter Data Frame
text_df = merged_df.reset_index(drop=True)
text_df["reviewText"] = text_df["reviewText"].astype(str)
text_df.head(10)

,reviewText,asin,overall,unixReviewTime,brand,title,text_len,year
0,What a spectacular tutu! Very slimming.,0000032034,5.0,2015-06-03,BubuBibi,Adult Ballet Tutu Yellow,6,2015
1,What the heck? Is this a tutu for nuns? I know...,0000032034,1.0,2015-04-01,BubuBibi,Adult Ballet Tutu Yellow,39,2015
2,Exactly what we were looking for!,0000032034,5.0,2015-01-13,BubuBibi,Adult Ballet Tutu Yellow,6,2015
3,I used this skirt for a Halloween costume and ...,0000032034,5.0,2014-12-23,BubuBibi,Adult Ballet Tutu Yellow,84,2014
4,This is thick enough that you can't see throug...,0000032034,4.0,2014-12-15,BubuBibi,Adult Ballet Tutu Yellow,28,2014
5,Arrived on time. Nice Atlas - Easy to read.,0899332757,5.0,2016-12-27,Garmin,Delorme New York State Atlas &amp; Gazetteer,9,2016
6,More of a road map than a useful topographical...,0899332757,3.0,2016-12-12,Garmin,Delorme New York State Atlas &amp; Gazetteer,37,2016
7,Comprehensive atlas. Very happy with how much ...,0899332757,5.0,2016-08-13,Garmin,Delorme New York State Atlas &amp; Gazetteer,32,2016
8,Delorme has always made the best book maps in ...,0899332757,5.0,2016-08-02,Garmin,Delorme New York State Atlas &amp; Gazetteer,14,2016
9,If you're interested in exploring any area of ...,0899332757,5.0,2016-08-01,Garmin,Delorme New York State Atlas &amp; Gazetteer,157,2016


### 5.1. Lowercasing

Mithilfe des Lowercasings werden alle Großbuchstaben in Kleinbuchstaben konvertiert, da diese für das Verständnis der Texte nicht benötigt werden. Hierfür wird zuerst eine Funktion definiert, welche einen string in Kleinbuchstaben konvertiert (Pomer, 2022).

In [ ]:
# Definition der Funktion, um alle Buchstaben in Kleinschreibung zu konvertieren
# Quelle: https://thecattlecrew.net/2022/08/03/textklassifikation-vorverarbeitung-der-daten/
def to_lower(in_string):
    out_string = in_string.lower()
    return out_string

In [ ]:
# Durchführen des Lowercasing mit Beschleunigung
text_df["text_lower"] = Parallel(n_jobs=-1)(delayed(to_lower)(in_string) for in_string in tqdm(text_df["reviewText"],
                                                                                                desc ="Status Lowercasing: ",
                                                                                                total = len(text_df)))

# Ausgabe
pd.concat([text_df[["reviewText", "text_lower"]]], axis=1, sort = False, keys = ["Lowercasing"])

Status Lowercasing:   0%|          | 0/2264641 [00:00<?, ?it/s]

Lowercasing  \
                                                reviewText   
0                  What a spectacular tutu! Very slimming.   
1        What the heck? Is this a tutu for nuns? I know...   
2                        Exactly what we were looking for!   
3        I used this skirt for a Halloween costume and ...   
4        This is thick enough that you can't see throug...   
...                                                    ...   
2264636                     Wife laughs at me but it works   
2264637  2XU makes great compression shorts. A bit more...   
2264638  These are my favorite compression shorts for r...   
2264639  Love the shorts.. amazing comfortable, perfect...   
2264640  I have a briley bolt release paddle installed ...   

                                                            
                                                text_lower  
0                  what a spectacular tutu! very slimming.  
1        what the heck? is this a tutu for nuns? i know...  
2                        exactly what we were looking for!  
3        i used this skirt for a halloween costume and ...  
4        this is thick enough that you can't see throug...  
...                                                    ...  
2264636                     wife laughs at me but it works  
2264637  2xu makes great compression shorts. a bit more...  
2264638  these are my favorite compression shorts for r...  
2264639  love the shorts.. amazing comfortable, perfect...  
2264640  i have a briley bolt release paddle installed ...  

[2264641 rows x 2 columns]

### 5.2. Lemmatisierung
Bei der Lemmatisierung handelt es sich um die Umwandlung von Wörtern in ihre Grundform unter Verwendung von Wörterbüchern (Pomer, 2022). Hierbei werden Pluralformen in Singularformen und unterschiedliche Zeitformen in das Präsenz umgewandelt (Yuzhe-Lu, 2018). Für die Lemmatisierung wird eine Funktion definiert, welche die Sätze in einzelen Token aufsplittet, dessen POS-Tags ermittelt und basierend dieser Tags die Lemmatisierung mithilfe der Funktion "lemmatize()" durchführt (Johnson, 2023).  

In [ ]:
# Definition der Funktion zur Lemmatisierung, um Wörter in ihre Grundform zu konvertieren
# Quelle 1: https://github.com/rrathgithub/Topic-Modeling-on-Amazon-Reviews-using-LDA/blob/master/2_LDA_Data_Processing.ipynb
# Quelle 2: https://www.machinelearningplus.com/nlp/lemmatization-examples-python/#stanfordcorenlplemmatization

def lemmatize(in_string):
    # Definition der notwendigen Parameter
    list_pos = 0                                  # Zuordnung einer Positionsnummer
    cleaned_str = ''                              # Leerer String für bereinigte Wörter
    text_token = nltk.word_tokenize(in_string)    # Tokenisieren der Sätze in einzelne Strings
    tagged_words = pos_tag(text_token)            # Grammatikalisches Tagging
    wnl = WordNetLemmatizer()                     # Klasse von NLTK für Lemmatisierung

    # Durchfürhung der Lemmatisation und Zusammenführung der Ergebnisse in einen String
    for word in tagged_words:
        if 'v' in word[1].lower():
            lemma = wnl.lemmatize(word[0], pos='v')
        else:
            lemma = wnl.lemmatize(word[0], pos='n')
        if list_pos == 0:
            cleaned_str = lemma
        else:
            cleaned_str = cleaned_str + ' ' + lemma
        list_pos += 1
    return cleaned_str

In [ ]:
# Durchführung der Lemmatisierung mit Beschleunigung
text_df["text_lemmatize"] = Parallel(n_jobs=-1)(delayed(lemmatize)(in_string) for in_string in tqdm(text_df["text_lower"],
                                                                                                    desc ="Status Lemmatisierung: ",
                                                                                                    total = len(text_df)))

# Ausgabe
pd.concat([text_df[["text_lower", "text_lemmatize"]]], axis=1, sort = False, keys = ["Lemmatisierung"])

Status Lemmatisierung:   0%|          | 0/2264641 [00:00<?, ?it/s]

Lemmatisierung  \
                                                text_lower   
0                  what a spectacular tutu! very slimming.   
1        what the heck? is this a tutu for nuns? i know...   
2                        exactly what we were looking for!   
3        i used this skirt for a halloween costume and ...   
4        this is thick enough that you can't see throug...   
...                                                    ...   
2264636                     wife laughs at me but it works   
2264637  2xu makes great compression shorts. a bit more...   
2264638  these are my favorite compression shorts for r...   
2264639  love the shorts.. amazing comfortable, perfect...   
2264640  i have a briley bolt release paddle installed ...   

                                                            
                                            text_lemmatize  
0                what a spectacular tutu ! very slimming .  
1        what the heck ? be this a tutu for nun ? i kno...  
2                            exactly what we be look for !  
3        i use this skirt for a halloween costume and g...  
4        this be thick enough that you ca n't see throu...  
...                                                    ...  
2264636                       wife laugh at me but it work  
2264637  2xu make great compression short . a bit more ...  
2264638  these be my favorite compression short for run...  
2264639  love the short .. amaze comfortable , perfect ...  
2264640  i have a briley bolt release paddle instal on ...  

[2264641 rows x 2 columns]

### 5.3. Stemming

Da die Lemmatisierung nicht alle Worte bereinigt hat, wie es zum Beispiel bei dem Wort "quickly" zu erkennen ist, welches eigentlich in "quick" konvertiert werden sollte, wird im folgenden zusätzlich der sogenannte Snowball-Stemming-Algorithmus implementiert. Im Gegensatz zur Lemmatisierung, beruft sich das Stemming nicht auf Wörterbücher, sondern entfernt die Sufixe und Präfixe der Wörter. Es findet daher kein Bezug auf den Kontext der Wörter statt (Luber & Litzel, Was ist Stemming?, 2020).

Zur Durchführung des Stemming wird eine Funktion definiert, welche die Sätze zuerst in einzelne Token unterteilt und den Snowball Stemmer auf diese mithilfe einer For-Schleife anwendet und wieder zu einem Satz verbindet (NLTK Project, 2023).

In [ ]:
# Definition der Funktion zur Durchführung des Stemmings
# Quelle: https://www.nltk.org/howto/stem.html
def stemming(in_string):
    # Definition der notwendigen Parameter
    text_token = nltk.word_tokenize(in_string)      # Tokenisieren der Sätze in einzelne Strings
    stemmer = SnowballStemmer("english")            # Laden der Klasse zum Stemming

    # Durchfürhung des Stemmings und Zusammenführung der Ergebnisse in einen String
    cleaned_str = ' '.join([stemmer.stem(word) for word in text_token])
    return cleaned_str

In [ ]:
# Durchführung des Stemmings mit Beschleunigung
text_df["text_stemming"] = Parallel(n_jobs=-1)(delayed(stemming)(in_string) for in_string in tqdm(text_df["text_lemmatize"],
                                                                                                    desc ="Status Stemming: ",
                                                                                                    total = len(text_df)))

# Ausgabe
pd.concat([text_df[["text_lemmatize", "text_stemming"]]], axis=1, sort = False, keys = ["Stemming"])

Status Stemming:   0%|          | 0/2264641 [00:00<?, ?it/s]

Stemming  \
                                            text_lemmatize   
0                what a spectacular tutu ! very slimming .   
1        what the heck ? be this a tutu for nun ? i kno...   
2                            exactly what we be look for !   
3        i use this skirt for a halloween costume and g...   
4        this be thick enough that you ca n't see throu...   
...                                                    ...   
2264636                       wife laugh at me but it work   
2264637  2xu make great compression short . a bit more ...   
2264638  these be my favorite compression short for run...   
2264639  love the short .. amaze comfortable , perfect ...   
2264640  i have a briley bolt release paddle instal on ...   

                                                            
                                             text_stemming  
0                    what a spectacular tutu ! veri slim .  
1        what the heck ? be this a tutu for nun ? i kno...  
2                              exact what we be look for !  
3        i use this skirt for a halloween costum and gl...  
4        this be thick enough that you ca n't see throu...  
...                                                    ...  
2264636                       wife laugh at me but it work  
2264637  2xu make great compress short . a bit more exp...  
2264638  these be my favorit compress short for run . t...  
2264639  love the short .. amaz comfort , perfect compr...  
2264640  i have a briley bolt releas paddl instal on my...  

[2264641 rows x 2 columns]

### 5.4. Entfernung von Satzzeichen

Nachdem die bereinigung der Wörter selbst abgeschlossen ist, werden nun die Satzzeichen aus dem Datensatz entfernt. Viele Satzzeichen verändern bei ihrem Fehlen den Kontext nicht groß, sodass diese problemlos bereinigt werden können. Jedoch ist zu beachten, dass beispielsweise Kommata oder Fragezeichen den Inhalt eines Textes verändern können, sodass es bei der Analyse zu Fehlinterpretationen kommt. Besonders zu beachten sind in diesem Zusammenhang Zahlen, da es bei dem Jahr "1998" und einem Geldwert "19,98" einen großen Unterschied macht, ob das Komma im Datensatz erhalten bleibt oder ob es verschwindet. Bei letzteren Fall hätten beide Zahlen den identischen Wert und das Ergebnis der Auswertung hätte keine Aussagekraft (Pomer, 2022). Aus diesem Grund soll im Folgenden mit diesem Wissen im Hinterkopf eine Bereinigung der Satzzeichen stattfinden. Treten bei dem Ergebnis der Analyse vermehrt Zahlen oder andere Unstimmigkeiten auf, müssen die Satzzeichen ggf. in die Modelle miteinbezogen werden.

Folgende Zeichen werden entfernt:

> '!"#$%&\'()*+,-./:;<=>?@[\\]^_{|}~'


In [ ]:
# Funktion zur Entfernung der Punktierung
# Quelle: https://www.geeksforgeeks.org/python-remove-punctuation-from-string/
def punctation(in_string):
    cleaned_string = in_string.translate(str.maketrans("","",string.punctuation))
    return cleaned_string

In [ ]:
# Bereinigung der Punktierung mit Beschleunigung
text_df["text_punctation"] = Parallel(n_jobs=-1)(delayed(punctation)(in_string) for in_string in tqdm(text_df["text_stemming"],
                                                                                                    desc ="Status bei der Entfernung der Satzzeichen: ",
                                                                                                    total = len(text_df)))

# Ausgabe
pd.concat([text_df[["text_stemming", "text_punctation"]]], axis=1, sort = False, keys = ["Punctation"])

Status bei der Entfernung der Satzzeichen:   0%|          | 0/2264641 [00:00<?, ?it/s]

Punctation  \
                                             text_stemming   
0                    what a spectacular tutu ! veri slim .   
1        what the heck ? be this a tutu for nun ? i kno...   
2                              exact what we be look for !   
3        i use this skirt for a halloween costum and gl...   
4        this be thick enough that you ca n't see throu...   
...                                                    ...   
2264636                       wife laugh at me but it work   
2264637  2xu make great compress short . a bit more exp...   
2264638  these be my favorit compress short for run . t...   
2264639  love the short .. amaz comfort , perfect compr...   
2264640  i have a briley bolt releas paddl instal on my...   

                                                            
                                           text_punctation  
0                      what a spectacular tutu  veri slim   
1        what the heck  be this a tutu for nun  i know ...  
2                               exact what we be look for   
3        i use this skirt for a halloween costum and gl...  
4        this be thick enough that you ca nt see throug...  
...                                                    ...  
2264636                       wife laugh at me but it work  
2264637  2xu make great compress short  a bit more expe...  
2264638  these be my favorit compress short for run  th...  
2264639  love the short  amaz comfort  perfect compress...  
2264640  i have a briley bolt releas paddl instal on my...  

[2264641 rows x 2 columns]

### 5.5. Entfernung von Stopwords

Stoppwörter können in Vorbereitung auf das Topic Modelling ebenfalls herausgefiltert werden, da diese keinen Einfluss auf die Bedeutung des Kontextes haben. Aufgrund ihrer Häufigkeit in einem Text wird darüber hinaus der Datensatz drastisch reduziert, wordurch die Laufzeit sowie die Genauigkeit der Analyse verbessert werden kann. Die Auswahl der Stopword muss jedoch bedacht getroffen werden, um die Ergebnisse nicht zu verfälschen. So ist beispielsweise die Negation "nicht" in vielen Sprachen als Stopword eingestuft, kann jedoch den Kontext eines Satzes bei dessen Entfernung dramatisch verändern (Teja, 2020). Für jede Sprache existieren unterschiedliche Stoppwörter. Um diese aus den Daten herauszufiltern werden sogenannte Stoppwordlisten herangezogen (Pomer, 2022).
Für die Bereinigung der Daten im Rahmen dieser Masterarbeit wird eine Liste der englischen Stopword der NLTK Library verwendet.

In [ ]:
# Definition der Funktion zur Entfernung der Stopwords
# Quelle: https://github.com/rrathgithub/Topic-Modeling-on-Amazon-Reviews-using-LDA/blob/master/2_LDA_Data_Processing.ipynb
def remove_stops(in_string):
    # Definition der notwendigen Parameter
    list_pos = 0                                  # Zuordnung einer Positionsnummer
    cleaned_str = ''                              # Leerer String für bereinigte Wörter
    text_token = nltk.word_tokenize(in_string)    # Tokenisieren der Sätze in einzelne Strings
    stop_words = stopwords.words('english')       # Bestimmen der Stopwords (für Englisch)
    stop_list = ["would", "could ", "also"]
    stop_words.append(stop_list)

    # Durchfürhung der Entfernung der Stopwords und Zusammenführung der Ergebnisse in einen String
    for word in text_token:
        if word not in stop_words:
          stop = word
        else:
          stop = ""
        if list_pos == 0:
            cleaned_str = stop
        else:
            cleaned_str = cleaned_str + ' ' + stop
        list_pos += 1

    return cleaned_str

In [ ]:
# Entfernung der Stopwords mit Beschleunigung
# Quelle (Fehlermeldung): https://github.com/scikit-learn-contrib/hdbscan/issues/494
text_df["text_stopwords"] = Parallel(n_jobs=1)(delayed(remove_stops)(in_string) for in_string in tqdm(text_df["text_punctation"],
                                                                                                    desc ="Status bei der Entfernung der Stopwords: ",
                                                                                                    total = len(text_df)))

# Ausgabe
pd.concat([text_df[["text_punctation", "text_stopwords"]]], axis=1, sort = False, keys = ["Stopwords"])

Status bei der Entfernung der Stopwords:   0%|          | 0/2264641 [00:00<?, ?it/s]

Stopwords  \
                                           text_punctation   
0                      what a spectacular tutu  veri slim    
1        what the heck  be this a tutu for nun  i know ...   
2                               exact what we be look for    
3        i use this skirt for a halloween costum and gl...   
4        this be thick enough that you ca nt see throug...   
...                                                    ...   
2264636                       wife laugh at me but it work   
2264637  2xu make great compress short  a bit more expe...   
2264638  these be my favorit compress short for run  th...   
2264639  love the short  amaz comfort  perfect compress...   
2264640  i have a briley bolt releas paddl instal on my...   

                                                            
                                            text_stopwords  
0                               spectacular tutu veri slim  
1          heck    tutu  nun  know   cut   still also  ...  
2                                           exact    look   
3         use  skirt   halloween costum  glue  bunch  f...  
4          thick enough   ca nt see      veri long   su...  
...                                                    ...  
2264636                                wife laugh     work  
2264637  2xu make great compress short  bit  expens    ...  
2264638     favorit compress short  run    thinnest mat...  
2264639  love  short amaz comfort perfect compress perf...  
2264640     briley bolt releas paddl instal   a400 xplo...  

[2264641 rows x 2 columns]

### 5.6. Entfernung von Zahlen

Im Rahmen der Masterarbeit wurde beschlossen, die Zahlen aus dem Datensatz zu entfernen, da die Annahme getroffen wurde, dass diese bei der Bewertung von Produkten keine signifikante Rolle spielen.
Mithilfe der Regular expressions werden die Nummern 0 bis 9 in einerm Parameter gespeichert. Dieser wird daraufhin mit dem Input Text abgeglichen und bei einer Übereinstimmung wird die Zahl mit "nichts" ausgetauscht (Python Software Foundation, 2023).

In [ ]:
# Definition der Funktion zur Entfernung der Zahlen
# Quelle: https://github.com/rrathgithub/Topic-Modeling-on-Amazon-Reviews-using-LDA/blob/master/2_LDA_Data_Processing.ipynb
# Quelle: https://thecattlecrew.net/2022/08/03/textklassifikation-vorverarbeitung-der-daten/
def remove_numbers(in_string):
    # Kompilieren der Zahlen 0-9 in ein Ausdrucksmuster
    num_re = re.compile('(\\d+)')

    # Ersetzen der Zahlen durch "nichts"
    cleaned_str = " ".join((re.sub(num_re, "", in_string)).split())
    return cleaned_str

In [ ]:
# Entfernung der Zahlen mit Beschleunigung
text_df["text_numbers"] = Parallel(n_jobs=-1)(delayed(remove_numbers)(in_string) for in_string in tqdm(text_df["text_stopwords"],
                                                                                                    desc ="Status bei der Entfernung der Zahlen: ",
                                                                                                    total = len(text_df)))

# Ausgabe
pd.concat([text_df[["text_stopwords", "text_numbers"]]], axis=1, sort = False, keys = ["Numbers"])

Status bei der Entfernung der Zahlen:   0%|          | 0/2264641 [00:00<?, ?it/s]

Numbers  \
                                            text_stopwords   
0                               spectacular tutu veri slim   
1          heck    tutu  nun  know   cut   still also  ...   
2                                           exact    look    
3         use  skirt   halloween costum  glue  bunch  f...   
4          thick enough   ca nt see      veri long   su...   
...                                                    ...   
2264636                                wife laugh     work   
2264637  2xu make great compress short  bit  expens    ...   
2264638     favorit compress short  run    thinnest mat...   
2264639  love  short amaz comfort perfect compress perf...   
2264640     briley bolt releas paddl instal   a400 xplo...   

                                                            
                                              text_numbers  
0                               spectacular tutu veri slim  
1        heck tutu nun know cut still also nt sever lay...  
2                                               exact look  
3        use skirt halloween costum glue bunch feather ...  
4        thick enough ca nt see veri long sure check di...  
...                                                    ...  
2264636                                    wife laugh work  
2264637  xu make great compress short bit expens well w...  
2264638  favorit compress short run thinnest materi run...  
2264639  love short amaz comfort perfect compress perfe...  
2264640  briley bolt releas paddl instal a xplor sport ...  

[2264641 rows x 2 columns]

### 5.7. Entfernung von nicht ASCII konvormen Wörtern

ASCII ist die Kurzform für „American Standard Code for Information Interchange“ und dient zur standartisierten Darstellung von Zeichen in elektronischer Form (IONOS SE, 2022). Zu beachten ist, dass es bei diesem um einen amerikanischen Standart handelt und daher keine Sonderzeichen, wie „ß“ oder „é, á“ berücksichtigt und daher beispielsweise bei deutschen Datensätzen das ergebnis verfälschen könnte (Pomer, 2022). Da der Amazondatensatz zufor auf die Sprache geprüft, ist bekannt, dass der Datensatz lediglich englische Sätze beinhaltet, wodurch der ASCII Standard verwendet werden kann.

Zur Bereinigung wird die untenstehende Funktion definiert, welche den Unicode der wörter ermittelt und diese mit den Unicodes aus dem ASCII Standard abgleicht. Lediglich die Wörter, welche zwischen 0 und 128 liegen werden im Datensatz beibehalten (Yuzhe-Lu, 2018).

In [ ]:
# Definition der Funktion zur Entfernung der ASCII characters
# Quelle: https://github.com/rrathgithub/Topic-Modeling-on-Amazon-Reviews-using-LDA/blob/master/2_LDA_Data_Processing.ipynb
def remove_non_ascii_words(in_string):
    # Returns the string without non ASCII characters
    cleaned_str = ''.join(word for word in in_string if 0 < ord(word) < 128)
    return cleaned_str

In [ ]:
# Entfernung der Stopwords mit Beschleunigung
text_df["text_ascII"] = Parallel(n_jobs=-1)(delayed(remove_numbers)(in_string) for in_string in tqdm(text_df["text_numbers"],
                                                                                                    desc ="Status bei der Entfernung der nicht ASCII vonformen Wörter: ",
                                                                                                    total = len(text_df)))

# Ausgabe
pd.concat([text_df[["text_numbers", "text_ascII"]]], axis=1, sort = False, keys = ["ASCII Characters"])

Status bei der Entfernung der nicht ASCII vonformen Wörter:   0%|          | 0/2264641 [00:00<?, ?it/s]

ASCII Characters  \
                                              text_numbers   
0                               spectacular tutu veri slim   
1        heck tutu nun know cut still also nt sever lay...   
2                                               exact look   
3        use skirt halloween costum glue bunch feather ...   
4        thick enough ca nt see veri long sure check di...   
...                                                    ...   
2264636                                    wife laugh work   
2264637  xu make great compress short bit expens well w...   
2264638  favorit compress short run thinnest materi run...   
2264639  love short amaz comfort perfect compress perfe...   
2264640  briley bolt releas paddl instal a xplor sport ...   

                                                            
                                                text_ascII  
0                               spectacular tutu veri slim  
1        heck tutu nun know cut still also nt sever lay...  
2                                               exact look  
3        use skirt halloween costum glue bunch feather ...  
4        thick enough ca nt see veri long sure check di...  
...                                                    ...  
2264636                                    wife laugh work  
2264637  xu make great compress short bit expens well w...  
2264638  favorit compress short run thinnest materi run...  
2264639  love short amaz comfort perfect compress perfe...  
2264640  briley bolt releas paddl instal a xplor sport ...  

[2264641 rows x 2 columns]

### 5.8. Selection relevanter Spalten für den Export

Nachdem alle Bereinigungen abgeschlossen sind, werden die relevanten Spalten selektiert und sind für den Export und die Analyse durch die Topic Modelling Modelle bereit.


In [ ]:
# Neuen Data Frame erzeugen, welcher nur die relevanten Spalten enhält
cleaned_data_df = text_df[["reviewText", "text_ascII", "year", "overall", "brand", "title", "asin"]]

# Rename colums
cleaned_data_df.rename(columns={"text_ascII": "cleanedText"}, inplace=True)

# Zurücksetzen des Index
cleaned_data_df.reset_index(inplace=True, drop=True)

# Ausgabe
cleaned_data_df.head(10)

<ipython-input-31-dd255f93ebd4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data_df.rename(columns={"text_ascII": "cleanedText"}, inplace=True)


,reviewText,cleanedText,year,overall,brand,title,asin
0,What a spectacular tutu! Very slimming.,spectacular tutu veri slim,2015,5.0,BubuBibi,Adult Ballet Tutu Yellow,0000032034
1,What the heck? Is this a tutu for nuns? I know...,heck tutu nun know cut still also nt sever lay...,2015,1.0,BubuBibi,Adult Ballet Tutu Yellow,0000032034
2,Exactly what we were looking for!,exact look,2015,5.0,BubuBibi,Adult Ballet Tutu Yellow,0000032034
3,I used this skirt for a Halloween costume and ...,use skirt halloween costum glue bunch feather ...,2014,5.0,BubuBibi,Adult Ballet Tutu Yellow,0000032034
4,This is thick enough that you can't see throug...,thick enough ca nt see veri long sure check di...,2014,4.0,BubuBibi,Adult Ballet Tutu Yellow,0000032034
5,Arrived on time. Nice Atlas - Easy to read.,arriv time nice atlas easi read,2016,5.0,Garmin,Delorme New York State Atlas &amp; Gazetteer,0899332757
6,More of a road map than a useful topographical...,road map use topograph map nt think would good...,2016,3.0,Garmin,Delorme New York State Atlas &amp; Gazetteer,0899332757
7,Comprehensive atlas. Very happy with how much ...,comprehens atlas veri happi much detail wish d...,2016,5.0,Garmin,Delorme New York State Atlas &amp; Gazetteer,0899332757
8,Delorme has always made the best book maps in ...,delorm alway make best book map usa three thumb,2016,5.0,Garmin,Delorme New York State Atlas &amp; Gazetteer,0899332757
9,If you're interested in exploring any area of ...,interest explor ani area new york beyond metro...,2016,5.0,Garmin,Delorme New York State Atlas &amp; Gazetteer,0899332757


## 6. Export der Bereinigten Daten

In [ ]:
# Schreiben eines hdf5 Files
# Quelle: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_hdf.html
cleaned_data_df.to_hdf('Cleaned_Sport_and_Outdoors.h5', key='df', mode='w')

## 7. Literaturverzeichnis

Erick. (12. 02 2018). How to unzip gz file using Python. Abgerufen am 24. 10 2022 von stackoverflow.com: https://stackoverflow.com/questions/31028815/how-to-unzip-gz-file-using-python

Free Software Foundation, I. (02. 04 2022). GNU Gzip: General file (de)compression. Abgerufen am 20. 02 2023 von gnu.org: https://www.gnu.org/software/gzip/manual/gzip.html

IONOS SE. (05. 04 2022). ASCII – Erklärung und Beispiele. Abgerufen am 08. 03 2023 von ionos.de: https://www.ionos.de/digitalguide/server/knowhow/ascii-american-standard-code-for-information-interchange/

Johnson, D. (21. 01 2023). POS-Tagging mit NLTK und Chunking in NLP [BEISPIELE]. Abgerufen am 21. 02 2023 von guru99.com: https://www.guru99.com/pos-tagging-chunking-nltk.html#1

Luber, D.-I. & Litzel, N. (30. 11 2020). Was ist Stemming? Abgerufen am 22. 02 2023 von bigdata-insider.de: https://www.bigdata-insider.de/was-ist-stemming-a-980852/#:~:text=Abgrenzung%20von%20Stemming%20und%20Lemmatisierung&text=Stemming%2DAlgorithmen%20arbeiten%20meist%20mit,)%2C%20um%20Stammformen%20zu%20finden.

NLTK Project. (02. 01 2023). Natural Language Toolkit - Sample usage for stem. Abgerufen am 22. 02 2023 von nltk.org: https://www.nltk.org/howto/stem.html

Pomer, L. (03. 08 2022). Textklassifikation – Vorverarbeitung der Daten. Abgerufen am 14. 02 2023 von thecattlecrew.net: https://thecattlecrew.net/2022/08/03/textklassifikation-vorverarbeitung-der-daten/

Python Software Foundation. (2023). re — Regular expression operations¶. Abgerufen am 08. 03 2023 von docs.python.org: https://docs.python.org/3/library/re.html

Teja, S. (10. 06 2020). Stop Words in NLP. Abgerufen am 24. 02 2023 von medium.com: https://medium.com/@saitejaponugoti/stop-words-in-nlp-5b248dadad47

Yuzhe-Lu. (10. 12 2018). Topic-Modeling-on-Amazon-Reviews-using-LDA/2_LDA_Data_Processing.ipynb. Abgerufen am 14. 02 2023 von https://github.com/rrathgithub/Topic-Modeling-on-Amazon-Reviews-using-LDA/blob/master/2_LDA_Data_Processing.ipynb